In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")    # modify the path based on your notebook location

In [5]:
import pipit as pp
import pandas as pd
from pipit.partitions import Event, Partition

In [ ]:
trace = pp.Trace.from_otf2("/home/rakrish/trace-data/sw4lite-32/")
trace._match_mpi()

In [ ]:
trace.events

In [ ]:
# Create event graph (linked list)

# Visualise Event Graph
#from graphviz import Digraph

# Function to create a DOT representation of linked lists
def linked_lists_to_dot(start_event_ids, dict):
    dot = Digraph(comment='Process Event Graph')

    for i, se in enumerate(start_event_ids):
        current_node = dict[se]
        while current_node:
            dot.node(f"{current_node.event_id}_{current_node.event_name}", str(current_node.event_name) + ":" + str(current_node.event_id))
            if current_node.get_next_event():
                dot.edge(f"{current_node.event_id}_{current_node.event_name}", f"{current_node.get_next_event().event_id}_{current_node.get_next_event().event_name}")

            if current_node.event_name == 'MpiSend' and current_node.has_matching_event():
                dot.edge(f"{current_node.event_id}_{current_node.event_name}", f"{current_node.get_matching_event().event_id}_{current_node.get_matching_event().event_name}")
            current_node = current_node.get_next_event()

    return dot

# Create a DOT representation
#dot_representation = linked_lists_to_dot(start_event_ids, event_dict)
#dot_representation.attr(rankdir='LR')  
# Display the DOT representation in a Jupyter Notebook (optional)
#dot_representation

In [ ]:
# Create partition graph (i.e. merge send-receives and SCC)
start_partition_ids, partition_dict = get_partition_graph(trace)
print (start_partition_ids)


for paritition_id, partition in partition_dict.items():
    print (f"{partition.partition_id} - {partition.event_dict}")


In [ ]:
# Display initial partition graph (before SCC)

from graphviz import Digraph

# Function to create a DOT representation of linked lists
def visualise(start_ids, partition_dict, dot=None):
    visited = []
    def visualise_recursive(start_ids, partition_dict, dot):
        nonlocal visited
        if dot is None:
            dot = Digraph(comment='Partition Graph')

        for i, sp in enumerate(start_ids):
            current_partition = partition_dict[sp]
            if current_partition.partition_id in visited:
                return dot
            dot.node(f"{current_partition.partition_id}", str(current_partition.partition_id))
            visited.append(current_partition.partition_id)
            children_ids = current_partition.get_children()
            for child_id in children_ids:
                child_partition = partition_dict[child_id]
                dot = visualise_recursive([child_id], partition_dict, dot)
                dot.edge(f"{current_partition.partition_id}", f"{child_partition.partition_id}")
        return dot
    return visualise_recursive(start_ids, partition_dict, dot)

 # Sample linked lists

# Create a DOT representation
dot_representation = visualise(start_partition_ids, partition_dict)
dot_representation.attr(rankdir='LR')  
# Display the DOT representation in a Jupyter Notebook (optional)
dot_representation

In [ ]:
# merge SCC

components = Partition.tarjan_strongly_connected(partition_dict)
print (components)

In [ ]:
partition_dict = Partition.merge_strongly_connected_components(partition_dict, components)
for sid in start_partition_ids:
    if sid not in partition_dict:
        print ("Something is wrong")

In [ ]:
dot_representation = visualise(start_partition_ids, partition_dict)
dot_representation.attr(rankdir='LR') 
dot_representation

In [ ]:
# Leap partitions
def print_leaps(dag):
    for i, leap in enumerate(dag.leaps):
        if len(leap.partitions_ids) < 1:
            continue
        print('Leap', i)
        print('  ', leap.partitions_ids)

In [ ]:
# Create leaps
from pipit.partitions.leap import Partition_DAG, Leap

unique_processes = trace.events['Process'].unique()

# root_partitions = []
# for root_id in start_partition_ids:
#     root_partitions.append(partition_dict[root_id])
#     print(root_id)
#     print(partition_dict[root_id].partition_id)

print("Creating partition DAG")

dag = Partition_DAG(root_partitions, partition_dict, set(unique_processes))

print("Creating dag")

dag.create_dag()

print("calculating distance") 

dag.calculate_distance()

pring("creating leaps")

# dag.create_leaps()


In [ ]:
# Leaps before completion
print_leaps(dag)

In [ ]:
# Complete leaps
dag.complete_leaps(force_merge=False)

In [ ]:
# Leaps after completion
print_leaps(dag)

In [ ]:
# Global/local step assignment
dag.global_step_assignment()

In [ ]:
dag.calculate_lateness()
dag.calculate_differential_lateness()

# quick fix for now - this won't be needed once all operations are taken into account
df = dag.global_step_df
df.loc[df['EventName'] == 'ProgramBegin', 'Lateness'] = 0.0
df.loc[df['EventName'] == 'ProgramBegin', 'DiffLateness'] = 0.0

display(dag.global_step_df.head(10))

In [ ]:
def lateness_vis(df, coloring_field: str = 'EventName'):
    from bokeh.transform import linear_cmap
    from bokeh.models import ColumnDataSource, HoverTool
    import bokeh
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import ColumnDataSource
    from bokeh.transform import factor_cmap, dodge
    from bokeh.palettes import Spectral5

    output_notebook()

    p = figure(title = "Trace", sizing_mode="stretch_width", height=200, tools="hover") 
    source = ColumnDataSource(df)
    p.y_range.flipped = True  # Flip the y-axis
    # index_cmap = factor_cmap('EventName', 
    #                          palette=Spectral5, 
    #                          factors=sorted(df['EventName'].unique()))

    
    if coloring_field == 'EventName':
        index_cmap = factor_cmap('EventName', 
                                palette=Spectral5, 
                                factors=sorted(df['EventName'].unique()))

    else:
        metric = df[coloring_field]
        # average = metric.mean()
        # std_dev = metric.std()
        # high = average
        # high = average + (std_dev / 16)
        # low = average -  std_dev
        # low = max(low, 0)

        high = df[coloring_field].max()
        low = df[coloring_field].min()
        # pallete = bokeh.palettes.cividis(100)
        # pallete = bokeh.palettes.Plasma256
        # pallete = bokeh.palettes.Turbo256
        pallete = bokeh.palettes.RdYlGn11
        # pallete = bokeh.palettes.Greys256[::-1]
        index_cmap = linear_cmap(coloring_field, low=low, high=high, palette=pallete)
        # print('min', df[coloring_field].min())
        # print('max', df[coloring_field].max())
        # print('low',low)
        # print('high',high)

    hover = HoverTool(tooltips=[("Process", "@Process"), ("Step", "@Step"), ("Lateness", "@Lateness")])
    p.add_tools(hover)  


    p.hbar(source=source,
        y="Process",
        right=dodge("Step", 1),
        left="Step",
        height=0.5,
        line_color="black",
        fill_color=index_cmap)

    p.segment(source=source,
            y0="Process",
            x1=dodge("Matching Step", 0.5),
            x0=dodge("Step", 0.5),
            y1="Matching Process", 
            color="black")
    p.yaxis.ticker = bokeh.models.FixedTicker(ticks=list(df['Process'].unique()))


    show(p) 

In [ ]:
lateness_vis(df)

In [ ]:
lateness_vis(df, coloring_field='Lateness')

In [ ]:
lateness_vis(df, coloring_field='DiffLateness')